In [13]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Extract text from the specified PDF
text = extract_text_from_pdf('infos/pdf2.pdf')
print(text)

EN   EN 
 
 
 EUROPEAN  
COMMISSION   
Brussels, 21.4.2021  
COM(2021) 206 final  
2021/0106 (COD)  
 
Proposal for a  
REGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL  
LAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE 
(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION 
LEGISLATIVE ACTS  
{SEC(2021)  167 final}  - {SWD(2021)  84 final}  - {SWD(2021)  85 final}   EN 1  EN EXPLANATORY MEMORANDUM  
1. CONTEXT  OF THE  PROPOSAL  
1.1. Reasons for and objectives of the proposal  
This explanatory memorandum accompanies the proposal for a Regulation laying down 
harmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence 
(AI) is a fast evolving family of technologies that can bring a wide array of economic and 
societal benefits across the entire s pectrum of industries and social activities. By improving 
prediction, optimising operations and resource allocation, and personalising service delivery, 
the use of artificial intelli

In [23]:
import pdfplumber

def extract_all_tables_from_pdf(file_path):
    tables = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                # look if there's any empty cells
                cleaned_table = [row for row in table if any(cell for cell in row)]
                if cleaned_table:
                    tables.append(cleaned_table)
    return tables

tables = extract_all_tables_from_pdf('infos/pdf2.pdf')

#for each table
#the only issue here is that sometimes it see texts as tables
for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(table)

Table 1:
[[None, 'anonymisation or'], ['pseudonymisation of judicial decisions, documents or data, communication between', None], ['personnel,', None]]
Table 2:
[['Where a high-risk AI system related to products to which the legal acts listed in Annex II,'], ['section A, apply, is placed on the market or put into service together with the product'], ['manufactured in accordance with those legal acts and under the name of the product'], ['manufacturer, the manufacturer of the product shall take the responsibility of the compliance'], ['of the AI system with this Regulation and, as far as the AI system is concerned, have the same'], ['obligations imposed by the present Regulation on the provider.']]
Table 3:
[[None, 'or it is necessary for the'], ['exercise of the right to freedom of expression and the right to freedom of the arts and', None], ['sciences guaranteed in the Charter of Fundamental Rights of the EU, and subject to', None], ['appropriate safeguards for the rights and freedoms

In [18]:
def chunk_text_and_save(text, output_file, chunk_size=500):
    with open(output_file, 'w', encoding='utf-8') as file:  
        start = 0
        while start < len(text):
            end = start + chunk_size
            if end >= len(text):
                file.write(text[start:])
            else:
                # until the word is reached it end to not split words 
                while end < len(text) and text[end] != ' ':
                    end -= 1
                file.write(text[start:end] + '\n')  
                start = end + 1 
            start = end
        return text

# Chunk text and save it to a file
chunk_text_and_save(text, 'chunked_text.txt', chunk_size=500)


'EN   EN \n \n \n EUROPEAN  \nCOMMISSION   \nBrussels, 21.4.2021  \nCOM(2021) 206 final  \n2021/0106 (COD)  \n \nProposal for a  \nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL  \nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE \n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION \nLEGISLATIVE ACTS  \n{SEC(2021)  167 final}  - {SWD(2021)  84 final}  - {SWD(2021)  85 final}   EN 1  EN EXPLANATORY MEMORANDUM  \n1. CONTEXT  OF THE  PROPOSAL  \n1.1. Reasons for and objectives of the proposal  \nThis explanatory memorandum accompanies the proposal for a Regulation laying down \nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence \n(AI) is a fast evolving family of technologies that can bring a wide array of economic and \nsocietal benefits across the entire s pectrum of industries and social activities. By improving \nprediction, optimising operations and resource allocation, and personalising service delivery, \nthe us

In [45]:
def chunk_table_rows_and_save_all(tables, output_file, chunk_size=3):
    table_chunks = []
    with open(output_file, 'w', encoding='utf-8') as file:
        for table in tables:
            for i in range(0, len(table), chunk_size):
                chunk = table[i:i + chunk_size]
                table_chunks.append(chunk) 
                for row in chunk:
                    cleaned_row = [cell if cell is not None else '' for cell in row]
                    file.write(','.join(cleaned_row) + '\n')
                file.write('\n')
    return table_chunks  


In [46]:
# Call chunk_text_and_save
text_chunks = chunk_text_and_save(text, 'chunked_text.txt', chunk_size=500)
print("Text chunks:", text_chunks)

# Call chunk_table_rows_and_save_all
table_chunks = chunk_table_rows_and_save_all(tables, 'chunked_tables.csv', chunk_size=3)
print("Table chunks:", table_chunks)



Text chunks: EN   EN 
 
 
 EUROPEAN  
COMMISSION   
Brussels, 21.4.2021  
COM(2021) 206 final  
2021/0106 (COD)  
 
Proposal for a  
REGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL  
LAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE 
(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION 
LEGISLATIVE ACTS  
{SEC(2021)  167 final}  - {SWD(2021)  84 final}  - {SWD(2021)  85 final}   EN 1  EN EXPLANATORY MEMORANDUM  
1. CONTEXT  OF THE  PROPOSAL  
1.1. Reasons for and objectives of the proposal  
This explanatory memorandum accompanies the proposal for a Regulation laying down 
harmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence 
(AI) is a fast evolving family of technologies that can bring a wide array of economic and 
societal benefits across the entire s pectrum of industries and social activities. By improving 
prediction, optimising operations and resource allocation, and personalising service delivery, 
the use of artif

In [52]:
#embeding using BERT - First Try

from transformers import BertTokenizer, BertModel
import torch
import sqlite3
import numpy as np

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors='pt')
    outputs = model(**inputs)
    # Use the [CLS] token representation
    embeddings = outputs.last_hidden_state[0][0].detach().numpy()
    return embeddings


NameError: name '_C' is not defined